In [1]:
import numpy as np  # http://www.numpy.org
import matplotlib.pyplot as plt   # http://matplotlib.org
import math as M

# Simple, conceptual model of marginal sea

In [2]:
#### Constants #####
L_x = 1e6 #zonal extent [m]
L_y = 2e6 #meridional extent [m]
A = L_x*L_y #surface area of interior [m^2]
P = 2*L_x+2*L_y #Perimeter [m]

g = 9.81 #gravitational acceleration [m/s^2]
C_p = 4.2e3 #specific heat capacity of sea water [J/(kg K)]
#alpha = 1e-4 #thermal expansion coefficient [K g m^-3 C^-1] -> page 4846
c = 0.006 #bottom slope coefficient
s = 0.02 #average bottom slope around the perimeter of the marginal sea

#### Parameters ####
H = 1000 #sill height [m]
f_0 = 1.2e-4#Coriolis parameter [s^-1]
Gamma = 20 #Restoring strength [W/(m^2 C)]

T_1 = 8.87 #ingoing temperature [Celsius]
T_A = 2.8 #Atmospheric temperature [Celsius]
ro_0 = 1027 #density sea water [kg m^-3]
#h_1 =  #thickness upper layer [m]

#### functions ####
L = H/s #width of sloping topography of the boundary current [m]
epsilon = c*P/L #ratio of heat fluxed into interior by eddies compared to heat flux by advection into the basin in the inflowing boundary current [-]
#mu = A*Gamma*f_0/(alpha*g*C_p*H**2*(T_1-T_A)) #[-]


In [3]:
# Since we're not getting acceptable results for T_out and T_0
# try to fix some parameters and use values from the literature

mu = 0.15*epsilon # value for mu/epsilon retrieved from table 1 in Spall frist row
print('mu/epsilon =', mu/epsilon)

# calculate temperatures with fixed mu
T_0 = T_1 - (mu/epsilon)*(M.sqrt(1+2*epsilon/mu)-1)*(T_1 - T_A)
T_out = T_1 - 2*mu*((T_1-T_A)/(T_1-T_0)*(1+P*L/A)-1)*(T_1-T_A)
print('T_1=', T_1, ', T_out=', T_out)


mu/epsilon = 0.15
T_1= 8.87 , T_out= 6.573029381034166


In [6]:
# Now that we have workable values for the temperatures 
# we can try to find values for the paramters that are clearly not working

# alpha is clearly incorrect, by fixing W we can check it's value
W = 2.32e6
Delta = (T_out-T_0)/(T_1-T_0)
alpha = W*(4*ro_0*f_0)/(g*H**2*(T_1-T_0)*(1-Delta))
print('alpha=', alpha)

alpha= 0.050754577676224044


In [11]:
#### Two main equations #####

#Interior temperature [Celsius]
T_0 = T_1 - (mu/epsilon)*(M.sqrt(1+2*epsilon/mu)-1)*(T_1 - T_A)

#Outflowing temperature [Celsius]
T_out = T_1 - 2*mu*((T_1-T_A)/(T_1-T_0)*(1+P*L/A)-1)*(T_1-T_A)

T_0, T_out

(6.333402634099233, 6.573029381034166)

In [16]:
#### Follow up computations ####

#Velocity in boundary current [m/s]
V_1 = alpha*g*H*(T_1-T_0)/(2*ro_0*f_0*L)

#effective perimeter [m]
#P_e = ... #[m] -> Not sure if we have to use this, and then how to determine it?

#Atmospheric heat flux/meridional heat transport at the sill [W/m^2]
Q_star = A*(T_0 - T_A)*Gamma + P*L*Gamma*(T_1-T_A)

#Fraction of outflowing waters that include inflowing water [-]
Delta = (T_out-T_0)/(T_1-T_0)

#Downwelling/Medidional overturning strength at the sill [m^2/s]
W = alpha*(g*H**2*(T_1-T_0)*(1-Delta))/(4*ro_0*f_0)


In [15]:
print('V_1=', "{:.2e}".format(V_1), ', Q_star=', "{:.2e}".format(Q_star), ', W=', "{:.2e}".format(W))

V_1= 1.02e-01 , Q_star= 1.78e+14 , W= 2.32e+06
